# Adding two numbers with NchooseK

The NchooseK programming model works best with Boolean values.  However, because any non-negative integer can be represented by a sequences of Booleans, it is possible, albeit awkward, to use NchooseK to add numbers.  This tutorial shows how.

We start by importing the `nchoosek` package and its `z3` solver.  Feel free to change `z3` to `ocean` to run on a D-Wave quantum annealer (assuming you have an account and have locally configured D-Wave Ocean).

In [1]:
import nchoosek
from nchoosek.solve import z3

We need to choose two numbers to add and specify a sufficient number of bits to represent each number.  Let's add 2 + 3, which requires two bits (10<sub>B</sub> + 11<sub>B</sub>).  We will be computing the sum with one additional bit.

In [2]:
num1 = 2
num2 = 3
nbits = 2

Every NchooseK program needs an "environment" in which to work.  (A program can employ more than one environment, but this is rarely needed.)  An environment is created by instantiating the `Environment` class: 

In [3]:
env = nchoosek.Environment()

The core piece of our addition program will be a <a href="https://en.wikipedia.org/wiki/Adder_(electronics)#Full_adder">full adder</a>.  A full adder inputs three bits—two single-bit values and a carry from a previous sum—and outputs two bits—a sum and a carry.  Full adders can be chained together to add multi-bit numbers in much the same way that one would add multi-digit decimal numbers with pencil and paper.

But how can we express a full adder using NchooseK?  The `tt2nck` helper program that comes with NchooseK can express any truth table as an NchooseK primitive.  Here's the truth table for a full adder, in which the columns represent, respectively, {carry in, A, B, carry out, sum}:
```
0 0 0 0 0
0 0 1 0 1
0 1 0 0 1
0 1 1 1 0
1 0 0 0 1
1 0 1 1 0
1 1 0 1 0
1 1 1 1 1
```

Save that to a file, say `full-adder.tt`, and run `tt2nck` on it:
```
$ tt2nck --compress full-adder.tt 
nck([A, B, C, D*2, E*3], {0, 4, 8})
```
(The `$` represents the shell prompt; don't type that.)

Here's an illustration of the preceding NchooseK primitive, with A–E replaced with more meaningful names:



You can verify that the primitive corresponds exactly to the rows of the full-adder truth table shown above.  For example, adding 1 + 1 with a carry in of 0 to produce 0 and a carry out of 1 (i.e., 0 + 1 + 1 = 2) corresponds to the row `0 1 1 1 0`.  This expands to {0, 1, 1, 1, 1, 0, 0, 0} when C<sub>out</sub> is repeated twice and ∑ is repeated three times.  The total number of 1 bits is four, and this is indeed one of the valid values of *k*.  In contrast, the invalid row `1 0 1 0 1` (1 + 0 + 1 = 1) expands to {1, 0, 1, 0, 0, 1, 1, 1}.  This contains five 1 bits, which is not one of the valid values of *k*.

To express the NchooseK full adder in Python we define a new NchooseK primitive type.  This will enable the primitive to be instantiated repeatedly.  The following type, called `adder` internally and referred to by the Python variable `Adder`, adds `carry-in`, `A`, and `B` and produces the two-bit value `C1` (high bit) and `C0` (low bit).  Note the nifty Python 3 shortcut for repeating `C1` twice and `C0` three times.

In [4]:
Adder = env.new_type('adder',
                     ['carry-in', 'A', 'B', 'C1', 'C0'],
                     nchoosek.Constraint(['carry-in', 'A', 'B', *2*['C1'], *3*['C0']], {0, 4, 8}))

NchooseK works with named "ports" rather than directly with variables.  We declare one port per input bit (maintained in `a` and `b` arrays), one per output bit (`c` array), and one per carry-out bit (`cout` array).

In [5]:
a = [env.register_port('A%d' % i) for i in range(nbits)]
b = [env.register_port('B%d' % i) for i in range(nbits)]
c = [env.register_port('C%d' % i) for i in range(nbits + 1)]
cout = [env.register_port('cout%d' % i) for i in range(nbits)]

For each bit of `num1`, we constrain the corresponding element of `a` to have the same value and likewise for `num2` and `b`.  To constrain some variable `x` to True, we require that one out of one of the variables in a list containing only `x` must be True.  To constrain `x` to False, we require that zero out of one of the variables in a list containing only `x` must be True.

In [6]:
for i in range(nbits):
    env.nck([a[i]], {(num1>>i)&1})
    env.nck([b[i]], {(num2>>i)&1})

The same approach lets us define a variable `always_false` that will be used for the initial carry in.

In [7]:
always_false = env.register_port('false')
env.nck([always_false], {0})

Each carry in is the previous carry out, with the first carry in being hard-wired to False.  We could define a new set of carry-in variables and constrain them to be equal to the associated carry out.  However, we will instead simply alias them in Python.  That is, our `cin` array will point to the same ports as our `cout` array, offset by one position.

In [8]:
cin = [always_false] + cout

It's finally time to connect all of the pieces created above!  The plan is to instantiate one `Adder` for each of the `nbits` bits we intend to add.

<table style="border: none">
    <tr>
        <td></td>
        <td></td>
        <td>cin<sub>N−1</sub></td>
        <td>⋯</td>
        <td>cin<sub>2</sub></td>
        <td>cin<sub>1</sub></td>
        <td style="border-style: dotted dotted none dotted; border-color: blue; border-width: medium">cin<sub>0</sub></td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td>A<sub>N−1</sub></td>
        <td>⋯</td>
        <td>A<sub>2</sub></td>
        <td>A<sub>1</sub></td>
        <td style="border-style: none dotted none dotted; border-color: blue; border-width: medium">A<sub>0</sub></td>
    </tr>
    <tr style="border-bottom: medium solid black">
        <td>+</td>
        <td></td>
        <td>B<sub>N−1</sub></td>
        <td>⋯</td>
        <td>B<sub>2</sub></td>
        <td>B<sub>1</sub></td>
        <td style="border-style: none dotted none dotted; border-color: blue; border-width: medium">B<sub>0</sub></td>
    </tr>
    <tr>
        <td></td>
        <td>C<sub>N</sub></td>
        <td>C<sub>N−1</sub></td>
        <td>⋯</td>
        <td>C<sub>2</sub></td>
        <td>C<sub>1</sub></td>
        <td style="border-style: none dotted none dotted; border-color: blue; border-width: medium">C<sub>0</sub></td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td>cout<sub>N−1</sub></td>
        <td>⋯</td>
        <td>cout<sub>2</sub></td>
        <td>cout<sub>1</sub></td>
        <td style="border-style: none dotted dotted dotted; border-color: blue; border-width: medium">cout<sub>0</sub></td>
    </tr>
</table>

Recall that we previous constrained cin<sub>0</sub> to False (0) and cin<sub><i>i</i></sub> to cout<sub><i>i</i>−1</sub>.  In the following code, after we instantiate all of the `Adder`s we additionally constrain C<sub><i>N</i></sub> to cout<sub><i>N</i>−1</sub> (= cin<sub><i>N</i></sub>).

In [9]:
for i in range(nbits):
    Adder([cin[i], a[i], b[i], cout[i], c[i]])
env.same(c[nbits], cout[nbits - 1])

Our problem is now completely defined.  It's time to solve for all of the variables.

In [10]:
soln = z3.solve(env)

All port names are now associated with a `bool`.  Let's convert the list of `C` values to a single `int` for more human-friendly output.

In [11]:
num3 = 0
for i in range(nbits, -1, -1):
    num3 <<= 1
    num3 += int(soln['C%d' % i])
print('%d + %d = %d' % (num1, num2, num3))

2 + 3 = 5


There you go!  We've demonstrated that it's indeed possible to perform simple integer addition using NchooseK.  Clearly, NchooseK is the wrong tool for performing arithmetic.  However, despite being based on constraints applied to Boolean values, NchooseK is at least capable of carrying out operations that may at first seem impossible.

If nothing else, this document lays out an approach to applying NchooseK to a computational problem that can apply to other, more appropriate problems as well:

1. Create an NchooseK environment and define one NchooseK port per program variable.

2. Use `tt2nck` to help define NchooseK primitives.

3. Instantiate the primitives as needed, and associate ports with other ports (e.g., using `same`).

4. Solve the set of constraints, and output the results.